In [1]:
def center(x,q):
    if q % 1 == 0:
        qhalf = Integer((q-1)/2)
        
        return ((Integer(x) + qhalf) % q) - qhalf;
    else:
        return 0;

In [2]:
def find_n_cyclotomic_trinomial(lbits,hbits):
    set = [];
    for i in range(1,hbits+1):
        for j in range(0,hbits+1):
            n = 2^i*3^j;
            
            if n > 2^hbits or n < 2^lbits:
                continue;
            else:
                set.append(n);
    
    set.sort();
            
    return set;

In [3]:
def find_ntt_prime_cyclotomic(n,b,lbits,hbits):
    
    fac = factor(n);
    
    if len(fac) > 2 or len(fac) == 0:
        #print("does not support cyclotomic trinomial");
        return 0;

    if len(fac) == 1:
        if fac[0][0] != 2:
            #print("does not support cyclotomic trinomial");
            return 0;
            
    if len(fac) == 2:
        if fac[0][0] != 2 or fac[1][0] != 3:
            #print("does not support cyclotomic trinomial");
            return 0;
        
    if ((n % b) != 0) or (n/2 < b):
        #print("please select adequete b");
        return 0;
    
    w_order = 3*n/b;
    
    qs =[];

    k = 1;

    while True:
        q = w_order*k+1;

        k += 1;

        if q < 2^lbits:
            continue;
            
        if q > 2^hbits:
            break;

        if q in Primes():
            qs.append(q);

    return qs;

In [4]:
def find_generator(q):

    Zq = IntegerModRing(q);

    gs = range(1,q);

    for x in list(factor(q-1)):
        p = x[0];

        t = [];

        for g in gs:
            if Zq(g)^((q-1)/p) != 1:
                t.append(Zq(g));

        gs = t;

    return gs;

In [5]:
def find_w_order_cyclotomic_trinomial(n,b):

    return 3*n/b;

In [6]:
def find_w_cyclotomic_trinomial(n,q,b):

    w_order = find_w_order_cyclotomic_trinomial(n,b);
    
    k = Integer((q-1)/w_order);
    
    ws = [g^k for g in find_generator(q)];
    
    return sorted(list(set(ws)));

In [7]:
def gen_tree(n,q,b):
    
    Zq = IntegerModRing(q);
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    level = Radix2+Radix3;

    tree = zero_matrix(ZZ,level+1,n/b);
    tree[0,0] = find_w_order_cyclotomic_trinomial(n,b);

    zetas = [Integer(Zq(-2^32)) & 0xffffffff];

    qprime = Integer(q^-1 % 2^32);
    
    #Radix-2 NTT with Cyclotomic Polynomial
    tree[1,0] = tree[0,0] / 6;
    tree[1,1] = 5*tree[0,0] / 6;


    
    #Radix-3 NTT
    for l in range(1,Radix3+1):
        for i in range(2*3^(l-1)):
            tree[l+1,3*i  ] = tree[l  ,  i] / 3;
            tree[l+1,3*i+1] = tree[l  ,  i] / 3 + tree[0,0]/3;
            tree[l+1,3*i+2] = tree[l  ,  i] / 3 + 2*tree[0,0]/3;

            x = Zq(w)^(tree[l+1,3*i]) * (-2^32);
            x = Integer(x);
            x = (x * qprime) & 0xffffffff;
            zetas.append(x);
            x = Zq(w)^(tree[l+1,3*i]*2) * (-2^32);
            x = Integer(x);
            x = (x * qprime) & 0xffffffff;
            zetas.append(x);
            

    #Radix-2 NTT
    for l in range(Radix3+1,level):
        for i in range(2*3^(Radix3)*2^(l-(Radix3+1))):
            tree[l+1,2*i  ] = tree[l  ,  i] / 2;
            tree[l+1,2*i+1] = tree[l  ,  i] / 2 + tree[0,0]/2;

            x = Zq(w)^(tree[l+1,2*i]) * (-2^32);
            x = Integer(x);
            x = (x * qprime) & 0xffffffff;
            zetas.append(x);
            
    return tree;    

In [8]:
def print_zetas(zetas):
    
    l = len(zetas)
    str1 = "const int16_t zetas[%d] = {" % l
    print(str1)
    for i in range(ceil(l/8)-1):
        str = "\t";
        for j in range(7):
            str += "%5d, " %zetas[8*i+j];
        str += "%5d," %zetas[8*i+7];
        print(str);

    str = "\t";
    for j in range((l%8-1)%8):
        str += "%5d, " %zetas[8*ceil(l/8)-8+j];
    str += "%5d" %zetas[l-1];  
    print(str);    
    print("};");

In [9]:
def print_params(n,q,d,w):
    
    zetas = gen_zetas(n,q,b);
    print_zetas(zetas);

    Zq = IntegerModRing(q);
    qprime = Integer(q^-1 % 2^32);
    
    print("====ntt====");
    x = Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3) * (-2^32);
    x = Integer(x);
    x = (x * qprime) & 0xffffffff;
    print("w %d" %x);

    print("====invntt====");
    print("w %d" %center(Integer(Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3)*2^16),q));
    z = Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/6);
    print("(z - z^5)^-1 %d" %center(Integer(Zq(2^16) * Zq(z - z^5)^-1),q))
    print("2^-1 %d" %center(Integer(Zq(2^-1 *2^16)),q))
    
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    print("level1 %d" %center(Integer(Zq(2^31*2^(-Radix2+1)*3^(-Radix3))),q))
    print("level2 %d" %center(Integer(Zq(2^32*2^(-Radix2+1)*3^(-Radix3))),q))
    print("====reduce.h====");    
    print("QINV : %d" %(q^-1 % 2^16))



In [10]:
n_set = find_n_cyclotomic_trinomial(9,11);
b_set = [1,2,3,4,6];

In [11]:
for n in n_set:
    print("n : ", n);
    for b in b_set:
        qs = find_ntt_prime_cyclotomic(n,b,0,15);
        if qs != 0:
            print("b : ", b, ", qs : ", qs);
    print("");

n :  512
b :  1 , qs :  [7681, 10753, 12289, 15361, 18433, 23041, 26113, 32257]
b :  2 , qs :  [769, 7681, 10753, 12289, 14593, 15361, 18433, 22273, 23041, 26113, 26881, 31489, 32257]
b :  4 , qs :  [769, 1153, 2689, 3457, 4993, 6529, 7297, 7681, 9601, 10369, 10753, 12289, 13441, 14593, 15361, 18049, 18433, 20353, 21121, 22273, 23041, 26113, 26497, 26881, 29569, 31489, 31873, 32257]

n :  576
b :  1 , qs :  [3457, 8641, 10369, 12097, 19009]
b :  2 , qs :  [2593, 3457, 8641, 10369, 12097, 16417, 19009, 21601, 25057, 28513, 30241]
b :  3 , qs :  [577, 1153, 3457, 6337, 7489, 8641, 10369, 12097, 13249, 14401, 18433, 19009, 20161, 21313, 23041, 26497, 27073, 30529, 32257]
b :  4 , qs :  [433, 1297, 2161, 2593, 3457, 3889, 6481, 8209, 8641, 10369, 12097, 15121, 16417, 17713, 19009, 19441, 21169, 21601, 23761, 25057, 28081, 28513, 30241, 32401]
b :  6 , qs :  [577, 1153, 2017, 2593, 3169, 3457, 6337, 7489, 8353, 8641, 8929, 10369, 10657, 12097, 13249, 13537, 14401, 16417, 16993, 17569, 18433

In [12]:
n = 576
q = 3457
b = 4
w = 81

In [13]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [14]:
find_w_order_cyclotomic_trinomial(n,b)

432

In [15]:
w = min(ws)

In [16]:
print_params(n,q,b,w)

NameError: name 'gen_zetas' is not defined

In [ ]:
Zq = IntegerModRing(q)
center(Integer(Zq(867)^-1*(2^16)),q)

In [ ]:
2^32

In [37]:
3396714084

3396714084

In [ ]:
const int16_t zetas[144] = {
	 2590, 3397956481, 2406523475, 1396454510, 1077158417, 1888443822, 559078763, 2477340119,
	1339304237, 660955338, 2139408066, 3273716756, 1129339101, 72059041, 2361797174, 1535603002,
	370234381, 2955663060, 195056369, 903222802, 708166433, 4162030791, 1490876701, 1623813207,
	200025958, 1610146838, 1410120880, 2395341900, 1434968825, 3334594222, 1212579717, 690772872,
	3773160451, 3714767780, 2292222928, 2872422444, 3856401067, 3027722101, 2904724773, 3550771343,
	3112205114, 122997328, 3484924289, 2015168341, 972797048, 1545542180, 735499173, 1042371294,
	900738007, 2067349026, 44726302, 1601450057, 2502188064, 2022622725, 213692328, 970312253,
	1456089578, 119270137, 332962464, 3809189971, 552866777, 587653900, 3966974422, 4004246340,
	262145820, 915646774, 1605177249, 2594125460, 2946966279, 3392986892, 703196844, 3942126477,
	100634178, 2529520803, 1554238961, 889556432, 2309616490, 3120901894, 356568011, 3342048605,
	3482439494, 255933834, 3199172921, 1566662934, 1915776561, 3455106755, 2386645119, 1822596767,
	1654873139, 3419077235, 4083759764, 788922255, 2666184501, 2167983203, 3477469905, 1032432116,
	1633752385, 1350485812, 1848687110, 3200415319, 7454384, 982736226, 1546784578, 4201787503,
	1443665606, 4207999489, 3207869702, 3429016413, 2914663951, 1997774780, 3497348261, 2221406285,
	1832535945, 355325614, 146602876, 715620817, 1282153963, 1526906222, 4053942230, 1152944649,
	129209315, 1285881155, 1041128897, 2679850870, 2117044916, 4219181064, 1200155745, 3601709631,
	1979138821, 1070946431, 501928490, 3178052168, 3096053949, 1572874920, 2481067310, 4248998598,
	1170338211, 3965732025, 2327010051, 2550641556, 3317200660, 3525923398, 1124369512, 1484664715
};